In [2]:
!pip install ta  # Thư viện tính technical indicators


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=f5ac54ee7e82c8217449cb3b65cfdbbd2bcf1c8b4ac2bd73b45cf9cfb2364d47
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [3]:
%%writefile /kaggle/working/RWKV-TS/configs/train_config.yaml

model:
  # === Input/Output ===
  enc_in: 22
  c_out: 1
  seq_len: 288
  pred_len: 12
  
  # === LSTM Architecture ===
  d_model: 64
  e_layers: 4
  dropout: 0.3

  # === Time Features ===
  time_features: 0  

data:
  path: "/kaggle/input/btcusdt-5m-full/BTCUSDT_5m_full.csv"
  freq: "5T"
  train_ratio: 0.8             # 80% training, 20% validation

training:
  epochs: 50                    #30
  batch_size: 128              # Increased batch size for GPU
  lr: 0.001                     # Reduced learning rate
  patience: 15                 # More patience for LSTM
  min_delta: 0.005             # More sensitive early stopping
  log_dir: "logs"
  checkpoint_dir: "checkpoints"
  device: "cuda"               
  clip_grad: 1.0               # Gradient clipping
  weight_decay: 0.0001         # L2 regularization
  resume: "auto"

Overwriting /kaggle/working/RWKV-TS/configs/train_config.yaml


In [4]:
!python /kaggle/working/RWKV-TS/src/train.py \
  --config /kaggle/working/configs/btc_5m_config.yaml \   

2025-05-11 09:34:39.486735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746956079.676142      60 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746956079.737332      60 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
lstm
Epoch 1/50: 100%|██████████| 4843/4843 [02:13<00:00, 36.41batch/s, loss=0.00047]
[Eval] MSE: 0.0662 | MAE: 0.1198 | RMSE: 0.2573 | SMAPE: 10.11% | R2: 0.8096
Epoch 2/50: 100%|█████████| 4843/4843 [02:11<00:00, 36.79batch/s, loss=0.000271]
[Eval] MSE: 0.0649 | MAE: 0.1162 | RMSE: 0.2548 | SMAPE: 8.09% | R2: 0.8134
Epoch 3/50: 100%|█████████| 4843/4843 [02:14<00:00, 36.13batch/s, loss=0.000176]
[Eval] MSE: 0.0622 | MAE: 0.1122 | R